# 01 - Introduction to SYCL Programming for GPUs

Argonne Leadership Computing Facility, UChicago Argonne, LLC, All rights reserved

In today's fast-paced world of HPC, **heterogenous systems** — which combine CPUs, GPUs, and other 
accelerators — are crucial for scaling computational workloads. However, programming for such diverse
architectures can be a challenge. This is where **SYCL** comes in.

**SYCL (pronounced "sickle")** is an open standard for single-source programming designed to simplify
development for heterogeneous systems. SYCL allows you to write **modern C++** code that runs on CPUs,
GPUs, and other accelerators without having to write separate code for each architecture. Whether you're
a seasoned developer of just starting, SYCL helps you focus on writing algorithms, while it handles the
underlying complexities of device management and memory allocation.
<!--
In the rapidly evolving world of computing, the ability to harness the
power of heterogeneous systems—where CPUs coexist with GPUs and other
accelerators—has become increasingly vital. **SYCL** stands as a
cutting-edge, single-source programming model designed to bridge this
gap. Developed to be used with modern C++, SYCL abstracts the
complexities associated with direct accelerator programming, making it
accessible to both novice and experienced developers.
-->

### What is SYCL?

SYCL is an open standard developed by the **Khronos Group**, the same group
responsible for **OpenCL**. It allows developers to write code for
heterogeneous systems using completely standard C++. This means that the
same code can target CPUs, GPUs, DSPs, FPGAs, and other types of
accelerators without modification. 

SYCL builds upon the foundation laid
by OpenCL, offering a higher level of abstraction and deeper integration
with C++. While OpenCL requires developers to manage host and device code separtely, SYCL allows both to be written in a single, unified C++ source file. This enables a more intuitive and efficient programming experience, making it easier to develop portabl and high-performance applications for diverse hardware platforms.

<img width="800" src=https://www.khronos.org/assets/uploads/apis/2022-sycl-diagram.jpg>
Image Source. https://www.khronos.org/sycl/

<img src=https://raw.githubusercontent.com/oneapi-src/oneAPI-samples/495ff2bb29b50698e9c6d3b12f7d8cf476e73d02/DirectProgramming/C++SYCL/Jupyter/oneapi-essentials-training/01_oneAPI_Intro/Assets/oneapi1.png>

### Advantages of SYCL

One of the primary advantages of SYCL is its ability to integrate
seamlessly with C++17 and upcoming versions, enabling features like
lambda functions, auto-typing, and templating. This integration not only
improves the programmability and readability of the code but also
leverages the **type safety** and **performance optimizations** provided by
modern C++. Here are a few key benefits: - 
* **Single-Source Development**: Unlike traditional approaches that might require maintaining separate code bases for different architectures (e.g., separate code for CPUs and GPUs), SYCL unifies the code into a **single source**. This simplifies development and reduces maintenance burdens, making it easier to write code that works across different devices without duplication.
* **Cross-Platform Portability**: SYCL code can be executed on any device that has a compatible SYCL runtime, providing true cross-platform capabilities. Whether you're working with **Intel GPUs, AMD GPUs, NVIDIA GPUs,** or even FPGAs, the same SYCL codebase can be compiled and executed, ensuring broad compatibility.
* **Performance**: With SYCL, developers do not have to sacrifice performance for portability. It allows fine control over **parallel execution** and **memory management**, which are critical for achieving optimal performance on GPUs and other accelerators. SYCL's abstraction ensures you can write high-level code without losing the ability to perform low-level optimizations when needed.

As GPUs continue to play a crucial role in fields ranging from
**scientific computing** to **machine learning**, mastering SYCL can provide
developers with the tools needed to fully exploit the capabilities of
these powerful devices. The following sections will guide you through
setting up your development environment, understanding the core concepts
of SYCL, and walking you through practical examples to kickstart your
journey in high-performance computing with SYCL.

<!--
------------------------------------------------------------------------

This introduction sets the stage for learning SYCL by highlighting its
relevance, advantages, and integration with modern C++. It aims to build
a strong foundation for the subsequent sections that delve deeper into
SYCL programming.

------------------------------------------------------------------------
-->

# Basics of a SYCL Kernel 
<!--
In SYCL, all computations are submitted through a queue. This queue is associated with a device, and any computation assigned to the queue is executed on this device[^1].
This is how we check if a gpu is available for use and then initialize a sycl queue for a gpu:
-->
In SYCL, computations are submitted to a **queue**, which is associated with a specific device (such as a CPU, GPU, or accelerator). The **queue** is the core abstraction that handles task submission and ensures that your kernels (functions that run on the device) are executed on the chosen hardware.

### Device Selection
One of SYCL's strengths is the ability to choose the device on which your code will execute. This selection is made through **device selectors**, which help determine whether the code runs on a CPU, GPU, or another accelerator.

Here's how you can check if a GPU is available and initialize a SYCL queue to run on that GPU:
```c++
// Check for available GPU devices
auto selector = sycl::gpu_selector{};           // Select a GPU device
auto myQueue = sycl::queue{selector};           // Create a queue for GPU
```
SYCL also provides other device selectors for different platforms:
* `sycl::default_selector_v`: Automatically selects the best available device (GPU if available, otherwise CPU or another device).
* `sycl::gpu_selector{}`: Specifically selects a GPU if present.
* `sycl::cpu_selector{}`: Selects a CPU for execution.
* `sycl::accelerator_selector{}`: Selects a specialized accelerator device like an FPGA.
  
Each selector is designed to give developers control over the type of hardware used for computations. If you dont' have a GPU or need to run the code on a CPU for testing purposes, the `cpu_selector{}` provides an easy fallback.

### Creating a Queue
After selecting a device, the next step is to create a **queue**. The queue manages the execution tasks (such as kernel functions) on the chosen device. Once a queue is created, you can submit tasks to it.
```c++
// Create a queue using the GPU selctor
auto myQueue = sycl::queue{sycl::gpu_selector{}};
```
Here, the queue is associated with a GPU. If no GPU is available, SYCL will throw an exception, which you can handle to provide a fallback, such as using the CPU.



# Understanding SYCL Kernel Command Group Execution

A **command group** is a fundamental construct that encapsulates a set of operations meant to be executed on a device. These operations can include tasks like kernel execution, memory management, or synchronization. The **command group** is submitted to a **queue**, and within this group, dependencies between tasks and data are managed to ensure that execution occurs in the correct order on the device.

### Submitting a Command Group

When you sumit work to a SYCL device, you do so through a **command group**. This is done using the `submit` function of the queue, which accepts a lambda function to define the operations you want to execute.

 ```c++
// Submit a command group to the queue
myQueue.submit([&](sycl::handler &cgh) {
  // Command Group Function:
  // Inside this lambda, we define the operations to be performed on the device
  // For example, kernel execution, data transfers, etc.
  // Lambda functions are explored further below
})
```
At the heart of the command group is the **command group handler (cgh)**, which acts as an intermediary between the host (CPU) and the device (GPU or other accelerators). The handler is used to:

* Define **kernel execution**: Specify the operations to be carried out on the device.
* Establish **data dependencies**: Ensure that the necessary data is available on the device when the kernel executes.
* Manage **memory accessors**: Allow access to buffers and other memory resources across host and device.

<img width="255" alt="" src="images/image11.png" >

> The diagram illustrates the process of defining and submitting a SYCL command group.
> It begins with a call to the `submit` function on a SYCL queue, which initiates the creation of a command group.
> The `submit` function takes a command group function as its argument, within which a command group handler `cgh` is created.
> * Inside the command group function, the handler is used to:
>   * Specify dependencies between tasks.
>   * Define the kernel funciton (the computation to be executed on the device).
>   * Set up accessors for memory objects that the kernel will use.
>
>Once these elements are defined, the command group is assembled and submitted for execution on the device.



# Enqueuing A Kernel

In SYCL, all computations are submitted through a queue. A queue is
associated with a device, and any computation assigned to the queue is
executed on this device. As a developer, you control the flow of computations by submitting **kernels** (parallel tasks) to the queue.

### Managing Data in SYCL

To efficiently execute kernels, SYCL provides two primary methods for managing data between the host (CPU) and device (GPU or other accelerators): 

1. **Buffer/Accessor Model:**
This Buffer/Accessor model is the traditional approach in SYCL and provides robust memory management and synchronization mechanisms. Buffers are used to store data, and accessors define how kernels access and manipulate this data.

The SYCL runtime automatically handles the transfer of data between the host and device, ensuring that data remain **consistent** and **synchronized** across different memory spaces.

Some key benefits include:

* **Automatic Memory Management**: SYCL takes care of copying data between host and device, ensuring the correct data is available at the right time.
*  **Data Consistency**: The runtime manages the synchronization of data across devices, which simplifies programmingin heterogenous environemnts

``` c++
sycl::buffer<float, 1> buf(data, sycl::range<1>(data_size));
myQueue.submit([&](sycl::handler &cgh) {
  // Access the buffer in read-write mode
  auto acc = buf.get_access<sycl::access::mode::read_write>(cgh);
  cgh.parallel_for(sycl::range<1>(data_size), [=](sycl::id<1> idx) {
    // Modify buffer data
    acc[idx] *= 2;
  });
});
```
* A **buffer** is created to hold the data.
* The kernel accesses the buffer using an **accessor**, ensuring safe access and synchronization.
* SYCL ensures that the correct data is copied to the device before the kernel runs and that modified data is transferred back to the host afterward.

2. **Unified Shared Memory (USM) Model:**
The USM model offers a more flexible way to manage memory, especially for developers who need fine control over memory allocation. With USM, you can allocate memory that is shared between the host and device, eliminating the need for explicit buffers and accessors. This simplifies certain types of programs, such as those where direct pointer manipulation is needed.

USM gives developers:

* **Simplified Memory Access**: USM allows the host and device to directly access the same memory locations, which can simplify memory management in some cases.
* **Fine-Grained Control**: Developers have more control over when and how memory is allocated, transferred, and synchronized between the host and device.

``` c++
float* usm_data = sycl::malloc_shared<float>(data_size, myQueue);
std::copy(data.begin(), data.end(), usm_data); // Copy data to shared memory

myQueue.submit([&](sycl::handler& cgh) {
  cgh.parallel_for(sycl::range<1>(data_size), [=](sycl::id<1> idx) {
    usm_data[idx] *= 2; // modify data Directly in shared memory
  });
});

myQueue.wait() // ensure the task is compelted
```
* Memory is allocated using `malloc_shared`, which allows both the host and device to directly access the same data.
* The kernel modifies the data in place, without the need for buffers or accessors.
* The use of `wait()` ensures that the kernel completes before the data is accessed on the host again.

# Scheduling

A schedulre is a component responsible for managing the order and
execution of tasks on computational resources.

#### Scheduling Overview
<img width="600" src="images/image33.png">

-   When the **submit** function is called, it creates a command group
    handler (**`cgh`**) and submits it to the scheduler.
-   The scheduler is responsible for executing the commands on the
    designated target device.

# Command Groups

A command group is a fundamental construct that encapsulates a set of
operations meant to be executed on a device.

<img width="305" src="images/image11.png" >



-   Command groups are defined by calling the **submit** function on the
    queue.
-   The **submit** function takes a command group handler (`cgh`) which
    facilitates the composition of the command group.
-   Inside the **submit** function, a handler is created and passed to
    the `cgh`.
-   This handler is then used by the `cgh` to assemble the command
    group.

``` c++
myQueue.submit([&](sycl::handler &cgh) {
  /* Command group function */
})
```

### Lambda functions 

In SYCL, lambdas play a crucial role similar to their use in general programming, but they are specifically tailored for defining operations on data that will be executed on parallel devices like GPUs and CPUs. Like in other programming contexts, lambdas in SYCL allow for writing concise, anonymous functions. This capability is especially valuable in SYCL due to the nature of parallel computing, where operations often need to be defined locally and executed across a range of data elements.

Lambdas in SYCL are structured similarly to standard C++ lambdas, but are specifically utilized within the SYCL framework to define the functionality of kernels that execute on parallel compute devices. The basic syntax of a lambda in SYCL can be summarized as follows:

```cpp
[capture_clause](input_signature) -> return_specification {
    // execution_block
}
```

In the context of SYCL you typically encounter the following types of captures:

- `[]` : Captures nothing from the enclosing scope. This is used when the lambda does not need to access any external variables.

- `[&]` : Captures all accessible variables from the surrounding scope by reference. Useful when you need to modify the external variables or when copying them is expensive.

- `[=]` : Captures all accessible variables from the surrounding scope by value. This is safe when the lambda is executed asynchronously or on a separate device, ensuring that it works with a consistent copy of the data.

For example, when defining a SYCL kernel, a developer might use a lambda to specify the computation that each thread should perform on the elements of a buffer. This lambda can capture necessary variables from its surrounding scope to use within the kernel execution:

```c++
buffer<float, 1> buf(data, range<1>(data_size));
myQueue.submit([&](handler& cgh) {
    auto acc = buf.get_access<access::mode::read_write>(cgh);
    cgh.parallel_for(range<1>(data_size), [=](id<1> idx) {
        acc[idx] *= 2; // Example operation: double each element
    });
});

```

## Enqueuing SYCL Kernel Function Single_task example

Let's walk through a simple example of enqueuing a single_task kernel in SYCL. The `single_task` kernel is one of the most straightforward ways to run a task on a device, perfect for situations where you only need to execute one operation without any parallelization. Think of it as a "hello world" for SYCL kernels.

We start by setting up the basics.
```c++
// Select GPU devices
auto gpu_selector = sycl::gpu_selector{};
auto myQueue = sycl::queue{gpu_selector};
```
Here, we’re creating a queue for a GPU device using the `sycl::gpu_selector`. If SYCL finds an available GPU on your system, this selector will make sure your code runs on it. If no GPU is available, you’d get an exception, which you could handle to gracefully fall back to a CPU or another device.

Now, moving on to submitting a task.
``` c++
myQueue.submit([&](sycl::handler &cgh) {
    // Create a stream for output within kernel
    auto os = sycl::stream{128, 128, cgh};
    // Execute a single task
    cgh.single_task([=]() {
      os << "Hello World!" << sycl::endl;
    });
    
}).wait(); // Wait for completion of gpuQueue
```
This is where the magic happens. We use the `submit` function to send a **command group** to the queue. The command group is where we define what the device (GPU, in this case) should actually do. Inside the command group, we use the **command group handler (cgh)** to tell the device what task we want to run.

You might notice we're using **sycl::stream** here. This is a handy tool in SYCL that lets you print from within the kernel. It’s not something you’ll use in every program, but it’s incredibly useful for debugging or when you want to output something directly from the device. In our case, it’s printing **"Hello World!"** from the GPU.

### Why Use `single_task`?

The **single_task** kernel, as its name suggests, runs exactly once. Unlike more complex kernels, which may run in parallel across multiple data points (more on that later), this kernel executes a single instance of the task. It’s a great starting point for writing simple kernels or testing device functionality.

In this case, we’ve used it to print `"Hello World!"`, but you could imagine using `single_task` to run any simple, non-parallel operation—like initializing device memory, performing a small computation, or writing a quick test to ensure your SYCL setup is working.

### Waiting for Completion

After submitting the task, we call `wait()`. This ensures that our program waits for the GPU to finish running the kernel before continuing. Without `wait()`, the CPU might move on to other tasks before the GPU has finished, which could cause some strange behavior in more complex programs.

**<font color="red">SEE example [00-hello.ipynb](examples/00-hello.ipynb)</font>**

# Managing Data

Above, we already touched on the differences between the **Buffer/Accessor Mode** and **Unified Shared Memory (USM)**. However, to really drive home their use cases, we'll use this section to dive into when you might choose one or the other. Both have their unique strengths, but the right choice depends on the complexity of your application and the level of control you need over data movement.

## Buffers & Accessors

The **Buffer/Accessor Model** is one of the most common ways to manage data in SYCL, and it handles a lot of the complexity for you. When you create a **buffer**, SYCL automatically manages data movement between the host (CPU) and device (GPU), ensuring that the correct data is in the right place at the right time.

<img width="600" src="images/image22.png">

The diagram above shows the relationship between **buffers**, **accessors**, and the device. Notice how buffers are created on the host, but the actual data may be needed on the device. SYCL takes care of the underlying memory transfers, optimizing when and how data is moved.

* **Buffers**: Buffers represent a block of data, such as an array or vector, which can be accessed by both the host and device. When a buffer is created, SYCL does not immediately allocate memory on the device. Instead, it waits until the data is needed, which minimizes unnecessary memory transfers.

* **Accessors**: Accessors are how kernels (the code running on the device) access the data stored in buffers. By requesting an access mode (e.g., read, write, or read/write), accessors ensure that memory is properly synchronized across devices. This is especially important when working with multiple devices or when ensuring data consistency between the host and device.

## Choosing Between Buffer/Accessor Model and USM

At this point, you might be wondering when to use **Buffers/Accessors** versus **USM (Unified Shared Memory)**. Both have their advantages, and the decision usually comes down to how much control you want over memory management.

**Use the Buffer/Accessor Model when:**

* You prefer **automatic memory management**. SYCL handles memory transfers between host and device for you, ensuring that data remains consistent and synchronized. This is ideal if you want SYCL to take care of the details and minimize manual data movement.
* You’re working with complex data dependencies. Accessors make it easy to specify when and how data is accessed, especially in multi-kernel or multi-device environments.
* You need **robust data synchronization**. SYCL ensures that data is automatically synchronized between host and device when using accessors, simplifying memory consistency across different devices.

**Use USM (Unified Shared Memory) when:**

* You need **fine-grained control** over memory. USM gives you more flexibility, allowing you to allocate memory directly on the host or device and share that memory between the two. This is great for advanced users who need to control exactly when data is copied or synchronized.
* You want to work with **pointers** and direct memory access. USM allows you to allocate shared or device-specific memory and then directly manipulate that memory with pointers, which can be more intuitive for some use cases, especially when porting existing code.

Here’s a quick analogy: think of **buffers/accessors** as a fully-managed service—SYCL takes care of everything for you. With **USM**, you’re doing the memory management yourself, but with more control.

Depending on your application, both approaches can be useful. For beginners or when working with more straightforward data transfers, **buffers/accessors** are often easier. But if you’re tuning for performance or handling more complex memory management, **USM** might give you the control you need.

**Examples Buffer/Accessor Model:**
```c++

std::vector<int> vectorA(N, 1);  // Vector A filled with 1s

// Buffers 
sycl::buffer<int> bufA {vectorA.data(),vectorA.size() };
// or
//auto bufA = sycl::buffer{vectorA.data(), sycl::range{N}};

// Accessor
sycl::accessor accA { bufA, cgh, sycl::read_only};

```

### Buffers

Explain HERE 

```c++
int const size = 10;
//  buffer is the memory object to transfer  data between host and device
buffer<int> A{ size };
// cgh is a handler that defines the command group which contains the task function
myQueue.submit([&](sycl::handler &cgh) {
    // accessor object allows access the buffer elements
    sycl::accessor accA { bufA, cgh};
};

// host_accessor allows the host to access the buffer memory
sycl::host_accessor result(A);  
```

## **Unified Shared Memory (USM) Model:** 
This model allows for direct data sharing between the host and device, simplifying memory management by eliminating the need for explicit buffers and accessors. Here is the following changes from the buffer/accessor model to USM model:

```c++
// Allocate memory using USM
 float* usmA = sycl::malloc_shared<float>(N, gpuQueue);

 // Initialize USM memory
 std::copy(vectorA.begin(), vectorA.end(), usmA);
```

# How to compile SYCL code

Now that we've talked about writing SYCL kernels and managing their memory, let's look at how to actually compile and run them. If you're working with **Intel's DPC++ (Data Parallel C++) compiler**, which is part of Intel's oneAPI toolkit, the typical command for compiling SYCL code looks like this:

```bash
icpx -fsycl compute.cpp -o ./a.out
```

Let’s break this down a little so you understand what each part of the command is doing:

* `icpx`: This is the **DPC++ compiler** command. It's based on the Intel compiler and is designed to work with SYCL code. If you’ve used **g++** or **clang++** for compiling regular C++ code, this will feel familiar.
* `-fsycl`: This flag tells the compiler that you’re working with SYCL. It enables the compilation of SYCL kernels and ensures the compiler can target different devices (like GPUs and CPUs). Without this flag, the compiler would just treat your code as regular C++.
* `compute.cpp`: This is the source file we’re compiling. In this case, it contains the SYCL kernel we’ve written.
* `-o ./a.out`: This option specifies the output file. After compiling, your program will be saved as `a.out`, which you can then run.

### Behind the Scenes

When you compile SYCL code, the compiler does more than just translate the C++ into machine code. It’s also figuring out how to split the code between the **host** (your CPU) and the **device** (your GPU or other accelerators). The `-fsycl` flag tells the compiler to handle both sides of the equation—compiling the parts that run on the host, while also generating the necessary device code for the GPU or accelerator.

### Running Your Program

Once your code is compiled, you can run it just like any other program:

```bash
./a.out
```

If everything is set up correctly, this will execute your SYCL code, running your kernels on whatever device you’ve selected—whether that’s a CPU, GPU, or something else. The oneAPI runtime will take care of selecting the right platform and device, or you can specify one directly using the device selectors we talked about earlier.

### TODO


### Parallel_for

Explain HERE 

```c++
myQueue.submit([&](sycl::handler &cgh) {
    sycl::accessor accA { bufA, cgh, sycl::write_only};
    cgh.parallel_for(N, [=](auto idx) { 
        accA[i] = idx });
    });
```